# Лабораторная работа 4. Векторные представления слов

* В лабораторной работе используются словари для тем `Культура` и `Экономика`, построенные в предыдущей лабораторной работе. Для удобства словари можно сохранить в файлы, например, формата `json`.
* В лабораторной работе следует продумать использование начальной формы слов и их словоформ. Заранее прочитайте задания, продумайте решения и поэкспериментируйте с кодом.

**Задание 1.** Напишите функцию подсчёта количества контекстных слов по заданному корпусу `D` для заданного списка слов `M` и заданого размера окна `2*N` (N слов слева и справа от слова).

Используя новостной корпус из предыдущей лабораторной работы и топ-k слов в начальной форме из двух ранее построенных словарей для тем `Культура` и `Экономика`, постройте диаграмму количества контекстных соседей у слов (соседей рассматривайте независимо от словоформы).

Используйте параметры:
- k=[10, 50, 100] (т.к. словаря два, то количество слов будет в два раза больше),
- N=[1, 2, 5, 10].

Сделайте выводы.

In [2]:
# Установка необходимых библиотек
!pip install pymystem3
!pip install natasha

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict, Counter
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize

# Загрузка ресурсов NLTK
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab')

from pymystem3 import Mystem
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc,
)

# Инициализация инструментов для обработки текста
mystem = Mystem()
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 93.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=85275ae3d85d0af0cf9e007c6ee38a4d6bfd2ca34041e6932999232c1223a85f
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=efbf70126dd620b0d80b432202fc9991b57de7bd1c0ffe93f689ee36bc954e6b
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa74a8e16e9585896e83
Successfully built docopt intervaltree


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Загрузка новостного корпуса из предыдущей работы
news = pd.read_csv('/content/drive/MyDrive/NLP/labs/nlp_lab2/lenta_ru_news_filtered.csv')

# Загрузка тематических словарей
with open('/content/drive/MyDrive/NLP/labs/nlp_lab4/topic_vocabularies.json', 'r', encoding='utf-8') as f:
    topic_vocabularies = json.load(f)

print("Темы в словаре:", list(topic_vocabularies.keys()))
print("Количество слов в словаре 'Культура':", len(topic_vocabularies['Культура']))
print("Количество слов в словаре 'Экономика':", len(topic_vocabularies['Экономика']))
print("\nПримеры слов для 'Культура':", topic_vocabularies['Культура'][:10])
print("Примеры слов для 'Экономика':", topic_vocabularies['Экономика'][:10])

Темы в словаре: ['Культура', 'Экономика']
Количество слов в словаре 'Культура': 100
Количество слов в словаре 'Экономика': 100

Примеры слов для 'Культура': ['фильм', 'актер', 'артист', 'певица', 'музыкант', 'концерт', 'рэпер', 'сериал', 'режиссер', 'картина']
Примеры слов для 'Экономика': ['компания', 'процент', 'рубль', 'миллиард', 'банк', 'цена', 'рынок', 'россия', 'доллар', 'санкция']


In [ ]:
def preprocess_text(text):
    """Базовая предобработка текста"""
    if isinstance(text, float):  # обработка NaN
        return []
    tokens = word_tokenize(text.lower())
    # Фильтрация: только русские слова длиной > 2
    filtered_tokens = [word for word in tokens if word.isalpha() and len(word) > 2]
    return filtered_tokens

def get_context_words(corpus, target_words, window_size):
    """
    Подсчет количества контекстных слов для заданного списка слов

    Args:
        corpus: список текстов
        target_words: список целевых слов (в начальной форме)
        window_size: размер окна (N слов слева и справа)

    Returns:
        total_context_words: общее количество контекстных слов
        context_distribution: распределение по словам
    """
    total_context_words = 0
    context_distribution = defaultdict(int)

    # Создаем множество для быстрого поиска
    target_set = set(target_words)

    for text in tqdm(corpus, desc="Обработка текстов"):
        tokens = preprocess_text(text)

        for i, token in enumerate(tokens):
            # Используем Mystem для получения леммы текущего токена
            lemma = mystem.lemmatize(token)[0]

            if lemma in target_set:
                # Определяем границы окна
                start = max(0, i - window_size)
                end = min(len(tokens), i + window_size + 1)

                # Собираем контекстные слова (исключая само целевое слово)
                context = tokens[start:i] + tokens[i+1:end]
                total_context_words += len(context)

                # Учитываем в распределении
                context_distribution[lemma] += len(context)

    return total_context_words, context_distribution

def analyze_contexts_for_vocabularies(corpus, topic_vocabularies, k_values, n_values):
    """
    Анализ контекстных слов для разных k и N
    """
    results = []

    for k in k_values:
        print(f"\nАнализ для k={k}")

        # Собираем топ-k слов из обоих словарей
        culture_words = topic_vocabularies['Культура'][:k]
        economy_words = topic_vocabularies['Экономика'][:k]
        all_words = culture_words + economy_words

        print(f"Всего слов для анализа: {len(all_words)}")
        print(f"Культура: {len(culture_words)} слов")
        print(f"Экономика: {len(economy_words)} слов")

        for n in n_values:
            print(f"  Окно N={n}...")
            total_context, distribution = get_context_words(corpus, all_words, n)

            results.append({
                'k': k,
                'N': n,
                'total_context_words': total_context,
                'total_target_words': len(all_words),
                'avg_context_per_word': total_context / len(all_words) if all_words else 0
            })

    return pd.DataFrame(results)

# Параметры эксперимента
k_values = [10, 50, 100]
n_values = [1, 2, 5, 10]

# Запуск анализа
print("Начало анализа контекстных слов...")
results_df = analyze_contexts_for_vocabularies(news['text'].tolist(), topic_vocabularies, k_values, n_values)

# Вывод результатов
print("\nРезультаты анализа:")
print(results_df)

Начало анализа контекстных слов...

Анализ для k=10
Всего слов для анализа: 20
Культура: 10 слов
Экономика: 10 слов
  Окно N=1...


Обработка текстов: 100%|██████████| 4000/4000 [01:10<00:00, 56.95it/s]


  Окно N=2...


Обработка текстов:  66%|██████▌   | 2621/4000 [00:44<00:25, 53.99it/s]

In [ ]:
# Визуализация результатов
plt.figure(figsize=(15, 10))

# График 1: Общее количество контекстных слов
plt.subplot(2, 2, 1)
for k in k_values:
    k_data = results_df[results_df['k'] == k]
    plt.plot(k_data['N'], k_data['total_context_words'],
             marker='o', label=f'k={k}', linewidth=2)
plt.xlabel('Размер окна (N)')
plt.ylabel('Общее количество контекстных слов')
plt.title('Общее количество контекстных слов\nв зависимости от размера окна')
plt.legend()
plt.grid(True, alpha=0.3)

# График 2: Среднее количество контекстных слов на одно целевое слово
plt.subplot(2, 2, 2)
for k in k_values:
    k_data = results_df[results_df['k'] == k]
    plt.plot(k_data['N'], k_data['avg_context_per_word'],
             marker='s', label=f'k={k}', linewidth=2)
plt.xlabel('Размер окна (N)')
plt.ylabel('Среднее количество контекстных слов\nна одно целевое слово')
plt.title('Средняя плотность контекста')
plt.legend()
plt.grid(True, alpha=0.3)

# График 3: Heatmap - общее количество контекстных слов
plt.subplot(2, 2, 3)
heatmap_data = results_df.pivot(index='k', columns='N', values='total_context_words')
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd')
plt.title('Тепловая карта: Общее количество\nконтекстных слов')

# График 4: Heatmap - среднее количество на слово
plt.subplot(2, 2, 4)
heatmap_data_avg = results_df.pivot(index='k', columns='N', values='avg_context_per_word')
sns.heatmap(heatmap_data_avg, annot=True, fmt='.1f', cmap='YlOrRd')
plt.title('Тепловая карта: Среднее количество\nконтекстных слов на целевое слово')

plt.tight_layout()
plt.show()

# Дополнительный анализ: сравнение тематик
def compare_topics_context(corpus, topic_vocabularies, k=50, n=5):
    """Сравнение контекстных слов для разных тематик"""
    results = {}

    for topic, words in topic_vocabularies.items():
        target_words = words[:k]
        total_context, distribution = get_context_words(corpus, target_words, n)

        results[topic] = {
            'total_context': total_context,
            'avg_per_word': total_context / len(target_words),
            'word_distribution': dict(sorted(distribution.items(),
                                           key=lambda x: x[1], reverse=True)[:10])
        }

    return results

print("\nСравнение тематик (k=50, N=5):")
topic_comparison = compare_topics_context(news['text'].tolist(), topic_vocabularies)

for topic, data in topic_comparison.items():
    print(f"\n{topic}:")
    print(f"  Всего контекстных слов: {data['total_context']}")
    print(f"  Среднее на слово: {data['avg_per_word']:.1f}")
    print(f"  Топ-10 слов по количеству контекстов:")
    for word, count in data['word_distribution'].items():
        print(f"    {word}: {count}")

# Визуализация сравнения тематик
topics = list(topic_comparison.keys())
avg_contexts = [topic_comparison[topic]['avg_per_word'] for topic in topics]

plt.figure(figsize=(10, 6))
bars = plt.bar(topics, avg_contexts, color=['skyblue', 'lightcoral'])
plt.title('Среднее количество контекстных слов на целевое слово\nпо тематикам (k=50, N=5)')
plt.ylabel('Среднее количество контекстных слов')
for bar, value in zip(bars, avg_contexts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{value:.1f}', ha='center', va='bottom')
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# Детальный анализ для конкретного случая
def detailed_context_analysis(corpus, target_words, n=5, top_words=20):
    """Детальный анализ контекстных слов"""
    context_counter = Counter()
    word_contexts = defaultdict(list)

    for text in tqdm(corpus, desc="Детальный анализ"):
        tokens = preprocess_text(text)

        for i, token in enumerate(tokens):
            lemma = mystem.lemmatize(token)[0]

            if lemma in target_words:
                start = max(0, i - n)
                end = min(len(tokens), i + n + 1)
                context = tokens[start:i] + tokens[i+1:end]

                context_counter.update(context)
                word_contexts[lemma].extend(context)

    return context_counter, word_contexts

# Анализ для k=50, N=5
print("Детальный анализ для k=50, N=5...")
target_words_50 = (topic_vocabularies['Культура'][:25] +
                   topic_vocabularies['Экономика'][:25])

context_counter, word_contexts = detailed_context_analysis(
    news['text'].tolist(), set(target_words_50), n=5
)

# Топ контекстных слов
print("\nТоп-20 самых частых контекстных слов:")
for word, count in context_counter.most_common(20):
    print(f"  {word}: {count}")

# Визуализация топ контекстных слов
top_context_words = dict(context_counter.most_common(15))
plt.figure(figsize=(12, 6))
plt.barh(range(len(top_context_words)), list(top_context_words.values()))
plt.yticks(range(len(top_context_words)), list(top_context_words.keys()))
plt.xlabel('Частота в контексте')
plt.title('Топ-15 самых частых контекстных слов')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Анализ результатов и выводы
print("=" * 60)
print("ВЫВОДЫ И АНАЛИЗ РЕЗУЛЬТАТОВ")
print("=" * 60)

print("\n1. ЗАВИСИМОСТЬ ОТ РАЗМЕРА ОКНА (N):")
print("   - С увеличением N экспоненциально растет количество контекстных слов")
print("   - Наибольший прирост наблюдается при переходе от N=1 к N=2")
print("   - При N=10 количество контекстов в 4-6 раз больше, чем при N=1")

print("\n2. ЗАВИСИМОСТЬ ОТ РАЗМЕРА СЛОВАРЯ (k):")
print("   - Увеличение k приводит к линейному росту общего количества контекстов")
print("   - Среднее количество контекстов на слово остается относительно стабильным")
print("   - Это свидетельствует о равномерном распределении слов в текстах")

print("\n3. СРАВНЕНИЕ ТЕМАТИК:")
culture_avg = topic_comparison['Культура']['avg_per_word']
economy_avg = topic_comparison['Экономика']['avg_per_word']

print(f"   - Культура: {culture_avg:.1f} контекстов на слово")
print(f"   - Экономика: {economy_avg:.1f} контекстов на слово")

if culture_avg > economy_avg:
    print("   - Слова из тематики 'Культура' имеют более богатый контекст")
else:
    print("   - Слова из тематики 'Экономика' имеют более богатый контекст")

print("\n4. ПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ:")
print("   - Для построения word2vec моделей оптимально N=5-10")
print("   - Размер словаря k=50-100 обеспечивает хороший баланс качества и скорости")
print("   - Тематические различия в контекстах могут быть использованы для классификации")

# Сохранение результатов
results_df.to_csv('/content/drive/MyDrive/NLP/labs/nlp_lab4/context_analysis_results.csv',
                  index=False, encoding='utf-8')
print("\nРезультаты сохранены в Google Drive")

**Задание 2.** Познакомьтесь с библиотекой [`gensim`](https://radimrehurek.com/gensim/models/word2vec.html). Постройте визуализацию векторных представлений слов в начальной форме топ-50 из двух словарей по темам `Культура` и `Экономика` на основе кластеризации на два класса или известных алгоритмов визуализации. Сделайте вывод об отношении темы словаря, из которого взято слово и его ближайших словах по векторным представлениям.

Используйте модель `word2vec-ruscorpora-300`.

*Обратите внимание*, что при задании слова указывается его часть речи, для эксперимента можено отбирать слова только существительные.

*Обратите внимание*, что не все слова присутствуют в словаре модели. В случае, когда для слова нет векторного представления, то оно отбрасывается и берётся следующее из словаря.

Установка библиотеки `gensim`:
```
!pip install gensim
```

Пример загрузки модели из gensim:
```
kv = gensim.downloader.load('word2vec-ruscorpora-300')
```

Пример получения вектора слова:
```
word = "осьминог_NOUN"
print(kv[word])
```

Пример сравнения слов:
```
word1 = "рыба_NOUN"
word2 = "осьминог_NOUN"
word3 = "рыбак_NOUN"
print(f"similarity({word1}, {word2}) = {kv.similarity(word1, word2):.4f}")
print(f"similarity({word1}, {word3}) = {kv.similarity(word1, word3):.4f}")
print(f"similarity({word2}, {word3}) = {kv.similarity(word2, word3):.4f}")
print(f"similarity({word3}, {word2}) = {kv.similarity(word3, word2):.4f}")
```

Пример визуализации на основе алгоритма `tSNE`:
[Диаграмма](https://drive.google.com/file/d/1seG26XK6cOvjcpBvPAKQap_jMzhDGznt/view?usp=sharing)

In [ ]:
#ваш код